In [25]:
import pandas as pd 
import numpy as np 

import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

# import working data

In [144]:
df = pd.read_csv('target.csv')

In [145]:
df = df.drop(columns= 'Unnamed: 0')

In [105]:
df = df.head(100)

In [147]:
df.reviews

0      ["Ok, before I start a few warnings. This will...
1      ['LMAAAOOOO! Thats Peeta folks! xDHahahahaha, ...
2      ['Finished this book about a month ago but it\...
3      ["Just to clarify: Yes, I did cry.I've read a ...
4      ['(A) 86% | ExtraordinaryNotes: It dwells on t...
                             ...                        
879    ['As Michael Kors once sighed to a clueless de...
880    ['Michael Pollan is a journalist, and an omniv...
881    ['“He was a great writer. He fell in love. It ...
882    ['Here\'s Blink in a nutshell:Split decisions ...
883    ["If you know me at all, you know I read a lot...
Name: reviews, Length: 884, dtype: object

# Let's Look at Data

In [106]:
df.describe()

,review_count,number_of_ratings,length,genre,rating
count,100.000000,1.000000e+02,1.000000e+02,0.0,100.000000
mean,43539.410000,1.152083e+06,1.850194e+09,NaN,4.064100
std,37213.623217,1.314679e+06,1.850193e+10,NaN,0.227446
min,210.000000,1.100500e+04,1.500000e+01,NaN,3.580000
25%,16597.500000,2.592898e+05,3.085000e+02,NaN,3.927500
50%,34179.000000,6.296500e+05,3.900000e+02,NaN,4.070000
75%,54708.500000,1.544170e+06,5.260000e+02,NaN,4.195000
max,192376.000000,7.878189e+06,1.850193e+11,NaN,4.620000


In [ ]:
target

In [107]:
# genre is null
df.isnull().value_counts()

title  summary  year_published  author  review_count  number_of_ratings  length  genre  rating  reviews  clean_titles  Target
False  False    False           False   False         False              False   True   False   False    False         False     100
dtype: int64

# let's clean summary

## Create a function to clean summary

In [108]:
df.summary


0     Harry has been burdened with a dark, dangerous...
1     Could you survive on your own in the wild, wit...
2     1970s Afghanistan: Twelve-year-old Amir is des...
3     Librarian's note: An alternate cover edition c...
4     It is the middle of the summer, but there is a...
                            ...                        
95    God is dead. Meet the kids.Fat Charlie Nancy's...
96    God Is Not Great makes the ultimate case again...
97    One choice can transform you—or it can destroy...
98    In Eats, Shoots & Leaves, former editor Lynne ...
99    The "complex and moving" (The New Yorker) nove...
Name: summary, Length: 100, dtype: object

In [109]:
# unicodedata.normalize removes any inconsistencies in unicode character encoding.
# encode to convert the resulting string to the ASCII character set. We'll ignore any errors in conversion, meaning we'll drop anything that isn't an ASCII character.
# .decode to turn the resulting bytes object back into a string.

In [119]:
def clean_article(df, col_name):
    cleaned_summaries = []
    for summary in df[col_name]:
        # Normalize the summary text and convert to lowercase
        cleaned_summary = unicodedata.normalize('NFKD', summary)\
            .encode('ascii', 'ignore')\
            .decode('utf-8', 'ignore')\
            .lower()
        cleaned_summaries.append(cleaned_summary)
    df[f'cleaned_{col_name}'] = cleaned_summaries

In [120]:
clean_article(df, 'summary')

/var/folders/d0/5dr5q0n145vcn3_m9dxy1xq80000gn/T/ipykernel_10962/4173208729.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'cleaned_{col_name}'] = cleaned_summaries


In [121]:
df.head(1)

,title,summary,year_published,author,review_count,number_of_ratings,length,genre,rating,reviews,clean_titles,Target,cleaned_summary
0,Harry Potter and the Deathly Hallows,"Harry has been burdened with a dark, dangerous...",2007,J.K. Rowling,78687,3437863,759,NaN,4.62,"[""Ok, before I start a few warnings. This will...",harry potter and the deathly hallows,unsuccessful,"harry has been burdened with a dark, dangerous..."


# Clean Reviews

In [150]:
clean_article(df,'reviews')

###  remove special chars and tokenize